In [3]:
import os
import skimage
from skimage import transform
from glob import glob
import pandas as pd
import numpy as np
from PIL import Image
from skimage import io, color
from skimage.transform import *
from sklearn import metrics
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D

## Data

In [4]:
disease_data = []
disease_class = []

In [5]:
def read_data(filepath, label):
    all_images = glob(filepath)
    print(len(all_images))
    for item in all_images:
        disease_class.append(label)
        image = skimage.io.imread(item)
        image = color.gray2rgb(image)
        image = skimage.transform.resize(image=image, output_shape=(64,64))
        disease_data.append(np.array(image))

In [7]:
class_paths = ['CNN_dataset/AbdomenCT/*.jpeg','CNN_dataset/ChestCT/*.jpeg','CNN_dataset/CXR/*.jpeg','CNN_dataset/Hand/*.jpeg','CNN_dataset/HeadCT/*.jpeg']

for i in range(len(class_paths)):
    read_data(class_paths[i], i)

10000
10000
10000
10000
10000


### Train Test Split

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(disease_data, to_categorical(np.array(disease_class)), test_size = 0.20)

## CNN

In [9]:
model = tf.keras.Sequential()
model.add(Conv2D(filters=64, kernel_size=(2,2),strides=(1, 1), padding='same', activation='relu', input_shape=(64,64,3))) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(Conv2D(filters=32, kernel_size=(2,2),strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 64)        832       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        8224      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 16, 16, 32)        0

### Compile

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Fit

In [11]:
model_log = model.fit(np.array(X_train), Y_train,
          batch_size=64,
          epochs=10,
          verbose=1,
          validation_split=.1)

Epoch 1/10
563/563 [==============================] - 149s 260ms/step - loss: 0.0867 - accuracy: 0.9727 - val_loss: 0.0252 - val_accuracy: 0.9967
Epoch 2/10
563/563 [==============================] - 149s 266ms/step - loss: 0.0120 - accuracy: 0.9968 - val_loss: 0.0209 - val_accuracy: 0.9948
Epoch 3/10
563/563 [==============================] - 154s 274ms/step - loss: 0.0075 - accuracy: 0.9978 - val_loss: 0.0073 - val_accuracy: 0.9975
Epoch 4/10
563/563 [==============================] - 158s 280ms/step - loss: 0.0058 - accuracy: 0.9984 - val_loss: 0.0131 - val_accuracy: 0.9975
Epoch 5/10
563/563 [==============================] - 151s 269ms/step - loss: 0.0057 - accuracy: 0.9983 - val_loss: 0.0044 - val_accuracy: 0.9990
Epoch 6/10
563/563 [==============================] - 151s 268ms/step - loss: 0.0029 - accuracy: 0.9990 - val_loss: 0.0106 - val_accuracy: 0.9980
Epoch 7/10
563/563 [==============================] - 150s 267ms/step - loss: 0.0066 - accuracy: 0.9981 - val_loss: 0.0075 -

### Evaluate

In [12]:
print("Accuracy: {:.2f}%".format(model.evaluate(np.array(X_test), np.array(Y_test))[1]*100))

313/313 [==============================] - 8s 25ms/step - loss: 0.0044 - accuracy: 0.9987
Accuracy: 99.87%
